In [1]:
import pandas as pd
import numpy as np

#daten einlesen
daten = pd.read_csv('daten_07_05_2020_bis_31_01_2023.csv',index_col="date")
daten = daten.sort_index(axis=0)

## Woche 2 - Funktionen und Risiko Kennzahlen

In der ersten Woche haben wir den SMA kennengelernt; leider mussten wir die Länge des betrachteten Intervall händisch anpassen.

Mit hilfe von Python Funktionen habt ihr die Möglichkeit eine anpassbare Schablone zu erstellen, welche euch die Arbeit deutlich erleichtert.

Von der Syntax her sind alle Funktionen gleich aufgebaut:

    def myFunction(INPUT):
      #beschreibung
      
      ... Berechnungen ...
      output = xyz

      return output

In [ ]:
def t1():
  #druckt beim Aufruf "HALLO!!" aus
  print("HALLO!!")

def t2(name:str):
  #name muss vom Format String sein
  #druckt beim Aufruf "HALLO," name aus
  print("HALLO,",name)

t1()

t2('Luca')

In [3]:
df = daten[["4a. close (EUR)"]].copy()

In [4]:
def sma(data: pd.DataFrame,intervall:int):
  #diese Funktion berechnet den SMA_intervall von dem Input data
  #der SMA wird immer von dem Column "4a. close (EUR)" berechnet
  #ACHTUNG: Stelle sicher, dass data immer diese Spalte besitzt
  spalten_name = "SMA_"+str(intervall)
  data[spalten_name] = data["4a. close (EUR)"].rolling(intervall).mean()


In [ ]:
#Erstelle SMA's der Länge 4,8,12,16,20
for i in [4,8,12,16,20]:
  sma(data=df, intervall = i)

df.head(20)

### Relativer Stärke Index

Der RSI schaut sich die letzten n Handelstage an und bildet das aritmetische Mittel von den positiven sowie negativen Renditen in dem Handelszeitraum. Der Quotient aus diesen nennt man $rs$.

#### $RSI(n) = 100 - (\frac{100}{1+rs})$


#### mit $rs = \frac{\frac{1}{n} ∑_{i=1}^n pd_i * 1_{pd_i>0}}{\frac{-1}{n} ∑_{i=1}^n pd_i * 1_{pd_i<0}}$  ,wobei $pd_i = p_i - p_{i-1}$

Beispiel:
| Tag | Preis | Preisdifferenz | 
| --- | --- | --- | 
| 0 | 100 | - | 
| 1 | 110 | +10 | 
| 2 | 105 | -5 | 
| 3 | 95 | -10 | 
| 4 | 115 | +20 | 
| 5 | 112 | -3 | 

->  $\frac{1}{n} ∑_{i=1}^n p_i * 1_{p_i>0} = 6$      (arithmetisches Mittel der positiven Preisdifferenzen)

->  $\frac{-1}{n} ∑_{i=1}^n p_i * 1_{p_i<0} = 3.6$      (arithmetisches Mittel der negativen Preisdifferenzen)

=>  $rs = \frac{15}{6} = 1.67$

Und somit ist der $RSI(5) = 100 - (\frac{100}{1+1.67}) = 62.5$

In [6]:
#Definition des Relative Strength Index (RSI)
def RSI_berechnen(data:pd.DataFrame,intervall:int):

  spalten_name = "RSI_"+str(intervall)
  data[spalten_name] = data["4a. close (EUR)"].rolling(intervall).mean()

  # Bestimme die Preisänderung zum jeweiligen Zeitpunkt t-1
  delta = data["4a. close (EUR)"].diff()

  # Get rid of the first row, which has NaN values
  delta = delta[1:]
  #print(delta)

  # Calculate the gains and losses
  up = delta.where(delta > 0, 0)
  down = -delta.where(delta < 0, 0)

  # Calculate the rolling average of the gains and losses
  #window_size = 14 #als default
  avg_gain = up.rolling(intervall).mean()
  avg_loss = down.rolling(intervall).mean()

  # Calculate the relative strength
  rs = avg_gain / avg_loss

  # Calculate the RSI
  data[spalten_name] = 100 - (100 / (1 + rs))

  return up,down

In [7]:
up,down = RSI_berechnen(df,20)

In [ ]:
df.tail(2)

In [9]:
# bestimme Portfolios und speichere diese in einem neuen DataFrame
def Portfolios(data:pd.DataFrame):
    #legt ein neues Portfolio an
    #nutzt SMA_n als einzigen Signalgeber
    #gibt ein DataFrame mit den täglichen PF Returns zurück
    
    df_output = data[["4a. close (EUR)"]].copy()
    df_output["pct"] = df_output["4a. close (EUR)"].pct_change().shift(-1)
    #df_output = df_output.dropna()

    #Signale bestimmen
    for i in df.filter(regex="SMA_").columns:
        print(i)
        #df_output["Sig_"+i] = df_output["4a. close (EUR)"] > data[i]    #ACHTUNG: Abfrage von zwei DF
        #df_output["PF_Ret_"+i] = df_output["Sig_"+i] * df_output["pct"]

        #in einer Zeile
        df_output["PF_Ret_"+i] = (df_output["4a. close (EUR)"] > data[i]) * df_output["pct"]

    
    df_output = df_output.dropna()      #entfernen der letzen Zeile, da diese noch keinen Return hat (quasi t+1)    
    return df_output

In [ ]:
myPFs = Portfolios(df)

In [15]:
#Risiko Kennzahlen
def risiko(data:pd.DataFrame):
  #das DataFrame sollte die Spalte "PF_Ret*" besitzen
  #in der PF_Ret* Spalte stehen die jeweilen Tagesrenditen der 
  nk = 4

  for k in data.filter(regex="PF_Ret_").columns:
    rf = 0.0
    pf_ret = np.round((data[k] +1).cumprod()[-1],nk)
    #pf_ret_ann = 
    pf_vol = np.round(np.sqrt(365)*data[k].std(),nk)
    pf_sha = np.round((pf_ret-rf)/pf_vol,nk)           #ACHTUNG: Returns müssen annulized sein
    pf_mdd = np.round((((data[k]+1).cumprod()/((data[k]+1).cumprod()).cummax() - 1.0).cummin()).min(),nk)
    pf_inv = (data[k]!=0.0).sum()                      #Anzahl investierte Tage
    pf_sgw = ((myPFs[k]!=0.0).diff().ne(0)).sum() -1   #Anzahl Signalwechsel     #-1, um den ersten Wechsel von NaN abzuziehen

    t_0 = data.index[0]
    t_N = data.index[-1]

    print(k,"\t", pf_ret, pf_vol, pf_sha, "maxDD = ", pf_mdd, pf_inv, pf_sgw, t_0, t_N)
  #Lege DataFrame an, mit:
  #     : Vola  : Return  : Sharpe : MaxDD : Inv. Tage : Anz. Signalwe :t_0         : t_N 
  # PF_1: 30%   : 27.54%  : 0.92   : -32%   : 300       : 99            : 01-05-2023 :  01-31-2023
  #       ...     ...     ...     ...     ...     ...     ...
  # PF_K: 30%   : 27.54%  : 0.92   : -32%   : 500       : 73            : 01-05-2023 :  01-31-2023

In [ ]:
risiko(myPFs)

In [13]:
#TODO I.
# 1. Lege DataFrame an und sichere die Daten in diesem die Daten von der Risiko Funktion
#    Skizze zu der Form des DataFrame:
  #     : Vola  : Return  : Sharpe : MaxDD : Inv. Tage : Anz. Signalwe :t_0         : t_N 
  # PF_1: 30%   : 27.54%  : 0.92   : 32%   : 300       : 99            : 01-05-2023 :  01-31-2023
  #       ...     ...     ...     ...     ...     ...     ...
  # PF_K: 30%   : 27.54%  : 0.92   : 32%   : 500       : 73            : 01-05-2023 :  01-31-2023

my_risk = pd.DataFrame(columns=["Return_ges","Vola_ann","SR","maxDD","PF_inv","PF_SigWechsel","t_0","t_N"])

In [14]:
#TODO II.
# 1. Verallgemeiner die Portfolio Funktion, sodass nicht nur SMA Signale
#    verwertet werden können.
# 2. Lege ein DataFrame mit den RSI Längen 8, 12, 16, 20 und filtere die
#    Performance nach dem größten Sharpe Ratio

### Ausblick für die dritte Woche
- Machine Learning Einführung (Regression & Decission Trees)
- Training- & Testdatensets
- RSI Vertiefung

### Ausblick vierte Woche
- OBV Indikator
- Bot fertigstellen

- eventuelle: Backtesting Paper schreiben